In [1]:
%tensorflow_version 1.x
!git clone https://github.com/tensorflow/cleverhans.git
!pip install cleverhans/
!pip install adversarial-robustness-toolbox

TensorFlow 1.x selected.
fatal: destination path 'cleverhans' already exists and is not an empty directory.
Processing ./cleverhans
  Created wheel for cleverhans: filename=cleverhans-3.0.1-cp36-none-any.whl size=253453 sha256=7329c8a2ba032abe906e1069c1330673850f00d1ee4201b9f2ad6ce3bf3fdacf
  Stored in directory: /tmp/pip-ephem-wheel-cache-7b2nu6of/wheels/d1/6b/1d/5cf7b3ca4c0cfc7f845628b8ed46366ab5f4f56b5483e9db7f
Successfully built cleverhans
  Found existing installation: cleverhans 3.0.1
    Uninstalling cleverhans-3.0.1:
      Successfully uninstalled cleverhans-3.0.1


In [2]:
  
from __future__ import absolute_import, division, print_function, unicode_literals
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import matplotlib.pyplot as plt
from art.attacks import FastGradientMethod
from art.classifiers import KerasClassifier
from art.utils import load_dataset
import tensorflow as tf
import tensorflow_hub as hub
tf.compat.v1.disable_eager_execution()

# Step 1: Load the CIFAR 10 dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("cifar10")) # Original Dataset
print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size) + "\n" +
      "y_train shape: " + str(y_train.shape) + "\n" + "y_train size: " + str(y_train.size) + "\n" +
      "x_test shape: " + str(x_test.shape) + "\n" + "x_test size: " + str(x_test.size) + "\n" +
      "y_test shape: " + str(y_test.shape) + "\n" + "y_test size: " + str(y_test.size) + "\n")

# Step 2: Load the victim model
hub_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
embed = hub.KerasLayer(hub_url, input_shape=(32,32,3) )
model = tf.keras.Sequential([
    embed,
    tf.keras.layers.Activation( 'softmax' )
])
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()
victim_classifier = KerasClassifier(model=model, clip_values=(0., 1.))

# Step 3: Evaluate the victim model on the benign dataset
predictions = victim_classifier.predict(x_test) # giving the classifier the x_test of the CIFAR-10 dataset.
accuracy_benign = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test) # calculates the accuracy of the predictions
print("Accuracy on benign test examples for victim classifier: {}%\n".format(accuracy_benign * 100))


Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
x_train size: 153600000
y_train shape: (50000, 10)
y_train size: 500000
x_test shape: (10000, 32, 32, 3)
x_test size: 30720000
y_test shape: (10000, 10)
y_test size: 100000

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________
Accuracy on benign test examples for victim classifier: 94.52000000000001%



In [3]:
    # Sampled dataset to train model
x_train, y_train = x_train[:5000], y_train[:5000] # take 5000 samples for the training set
print("x_train shape: " + str(x_train.shape) + "\n" + "x_train size: " + str(x_train.size) + "\n" +
      "y_train shape: " + str(y_train.shape) + "\n" + "y_train size: " + str(y_train.size) + "\n" +
      "x_test shape: " + str(x_test.shape) + "\n" + "x_test size: " + str(x_test.size) + "\n" +
      "y_test shape: " + str(y_test.shape) + "\n" + "y_test size: " + str(y_test.size) + "\n")

# Step 2: Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=x_train.shape[1:]))
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


# Step 3: Create the classifier
copycat_classifier = KerasClassifier(model=model, clip_values=(min_, max_))

x_train shape: (5000, 32, 32, 3)
x_train size: 15360000
y_train shape: (5000, 10)
y_train size: 50000
x_test shape: (10000, 32, 32, 3)
x_test size: 30720000
y_test shape: (10000, 10)
y_test size: 100000




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)       

In [4]:
from art.attacks import CopycatCNN
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=1000000) 
stolen_classifier = extraction_attack.extract(x=x_train[:2500], thieved_classifier=copycat_classifier)
# Step 4: Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
19/19 [==============================] - 10s 536ms/step - loss: 2.2505 - accuracy: 0.1320
Epoch 2/10
19/19 [==============================] - 10s 522ms/step - loss: 2.0305 - accuracy: 0.2418
Epoch 3/10
19/19 [==============================] - 10s 524ms/step - loss: 1.8325 - accuracy: 0.3438
Epoch 4/10
19/19 [==============================] - 10s 524ms/step - loss: 1.7780 - accuracy: 0.3396
Epoch 5/10
19/19 [==============================] - 10s 524ms/step - loss: 1.6843 - accuracy: 0.3803
Epoch 6/10
19/19 [==============================] - 10s 525ms/step - loss: 1.6462 - accuracy: 0.3840
Epoch 7/10
19/19 [==============================] - 10s 524ms/step - loss: 1.5777 - accuracy: 0.4256
Epoch 8/10
19/19 [==============================] - 10s 523ms/step - loss: 1.4849 - accuracy: 0.4535
Epoch 9/10
19/19 [==============================] - 10s 524ms/step - loss: 1.4234 - accuracy: 0.4873
Epoch 10/10
19/19 [==============================] - 10s 524ms/step - loss: 1.3562 - accura

In [5]:
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=1000000) 
stolen_classifier = extraction_attack.extract(x=x_train[:5000], thieved_classifier=copycat_classifier)
# Step 4: Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
39/39 [==============================] - 20s 522ms/step - loss: 1.4165 - accuracy: 0.4960
Epoch 2/10
39/39 [==============================] - 20s 522ms/step - loss: 1.3136 - accuracy: 0.5260
Epoch 3/10
39/39 [==============================] - 20s 523ms/step - loss: 1.2367 - accuracy: 0.5599
Epoch 4/10
39/39 [==============================] - 20s 521ms/step - loss: 1.1681 - accuracy: 0.5793
Epoch 5/10
39/39 [==============================] - 20s 525ms/step - loss: 1.0765 - accuracy: 0.6194
Epoch 6/10
39/39 [==============================] - 20s 521ms/step - loss: 1.0258 - accuracy: 0.6362
Epoch 7/10
39/39 [==============================] - 20s 523ms/step - loss: 0.9591 - accuracy: 0.6591
Epoch 8/10
39/39 [==============================] - 20s 522ms/step - loss: 0.8917 - accuracy: 0.6813
Epoch 9/10
39/39 [==============================] - 20s 524ms/step - loss: 0.8365 - accuracy: 0.7053
Epoch 10/10
39/39 [==============================] - 20s 523ms/step - loss: 0.7843 - accura

In [6]:
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=1000000) 
stolen_classifier = extraction_attack.extract(x=x_train[:15000], thieved_classifier=copycat_classifier)
# Step 4: Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
39/39 [==============================] - 20s 522ms/step - loss: 0.7196 - accuracy: 0.7484
Epoch 2/10
39/39 [==============================] - 20s 523ms/step - loss: 0.6598 - accuracy: 0.7780
Epoch 3/10
39/39 [==============================] - 20s 522ms/step - loss: 0.6033 - accuracy: 0.7937
Epoch 4/10
39/39 [==============================] - 20s 522ms/step - loss: 0.5743 - accuracy: 0.8063
Epoch 5/10
39/39 [==============================] - 20s 522ms/step - loss: 0.5090 - accuracy: 0.8269
Epoch 6/10
39/39 [==============================] - 20s 522ms/step - loss: 0.4823 - accuracy: 0.8379
Epoch 7/10
39/39 [==============================] - 20s 521ms/step - loss: 0.4495 - accuracy: 0.8442
Epoch 8/10
39/39 [==============================] - 20s 525ms/step - loss: 0.4058 - accuracy: 0.8582
Epoch 9/10
39/39 [==============================] - 20s 522ms/step - loss: 0.3650 - accuracy: 0.8736
Epoch 10/10
39/39 [==============================] - 20s 520ms/step - loss: 0.3321 - accura

In [7]:
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=2500) 
stolen_classifier = extraction_attack.extract(x=x_train[:2500], thieved_classifier=copycat_classifier)
# Step 4: Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
19/19 [==============================] - 10s 523ms/step - loss: 0.3065 - accuracy: 0.8890
Epoch 2/10
19/19 [==============================] - 10s 524ms/step - loss: 0.2792 - accuracy: 0.9025
Epoch 3/10
19/19 [==============================] - 10s 524ms/step - loss: 0.2309 - accuracy: 0.9215
Epoch 4/10
19/19 [==============================] - 10s 526ms/step - loss: 0.2246 - accuracy: 0.9264
Epoch 5/10
19/19 [==============================] - 10s 522ms/step - loss: 0.1960 - accuracy: 0.9387
Epoch 6/10
19/19 [==============================] - 10s 523ms/step - loss: 0.1780 - accuracy: 0.9416
Epoch 7/10
19/19 [==============================] - 10s 522ms/step - loss: 0.1736 - accuracy: 0.9428
Epoch 8/10
19/19 [==============================] - 10s 525ms/step - loss: 0.1684 - accuracy: 0.9449
Epoch 9/10
19/19 [==============================] - 10s 525ms/step - loss: 0.1462 - accuracy: 0.9494
Epoch 10/10
19/19 [==============================] - 10s 525ms/step - loss: 0.1626 - accura

In [8]:
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=5000) 
stolen_classifier = extraction_attack.extract(x=x_train[:5000], thieved_classifier=copycat_classifier)
# Step 4: Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
39/39 [==============================] - 20s 521ms/step - loss: 0.3186 - accuracy: 0.8924
Epoch 2/10
39/39 [==============================] - 20s 522ms/step - loss: 0.2514 - accuracy: 0.9195
Epoch 3/10
39/39 [==============================] - 20s 520ms/step - loss: 0.2418 - accuracy: 0.9167
Epoch 4/10
39/39 [==============================] - 20s 526ms/step - loss: 0.2235 - accuracy: 0.9215
Epoch 5/10
39/39 [==============================] - 20s 520ms/step - loss: 0.2187 - accuracy: 0.9267
Epoch 6/10
39/39 [==============================] - 20s 522ms/step - loss: 0.1853 - accuracy: 0.9375
Epoch 7/10
39/39 [==============================] - 20s 520ms/step - loss: 0.1830 - accuracy: 0.9409
Epoch 8/10
39/39 [==============================] - 20s 520ms/step - loss: 0.1924 - accuracy: 0.9299
Epoch 9/10
39/39 [==============================] - 20s 522ms/step - loss: 0.1847 - accuracy: 0.9355
Epoch 10/10
39/39 [==============================] - 20s 522ms/step - loss: 0.1547 - accura

In [9]:
extraction_attack = CopycatCNN(classifier=victim_classifier, batch_size_fit=128, batch_size_query=128, nb_epochs=10, nb_stolen=15000) 
stolen_classifier = extraction_attack.extract(x=x_train[:15000], thieved_classifier=copycat_classifier)
# Step 4: Evaluate the ART classifier on benign test examples
predictions = stolen_classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Epoch 1/10
39/39 [==============================] - 20s 520ms/step - loss: 0.1778 - accuracy: 0.9411
Epoch 2/10
39/39 [==============================] - 20s 519ms/step - loss: 0.1602 - accuracy: 0.9465
Epoch 3/10
39/39 [==============================] - 20s 522ms/step - loss: 0.1546 - accuracy: 0.9513
Epoch 4/10
39/39 [==============================] - 20s 521ms/step - loss: 0.1398 - accuracy: 0.9541
Epoch 5/10
39/39 [==============================] - 20s 521ms/step - loss: 0.1400 - accuracy: 0.9559
Epoch 6/10
39/39 [==============================] - 20s 521ms/step - loss: 0.1246 - accuracy: 0.9595
Epoch 7/10
39/39 [==============================] - 21s 526ms/step - loss: 0.1295 - accuracy: 0.9539
Epoch 8/10
39/39 [==============================] - 20s 519ms/step - loss: 0.1207 - accuracy: 0.9625
Epoch 9/10
39/39 [==============================] - 20s 521ms/step - loss: 0.1150 - accuracy: 0.9603
Epoch 10/10
39/39 [==============================] - 21s 530ms/step - loss: 0.1137 - accura